In [1]:
%load_ext sql
#%config SqlMagic.autopandas=False

In [2]:
%sql mysql://root@127.0.0.1/current

u'Connected: root@current'

### Join relevant tables
Join PDD to deliveries

In [15]:
%%sql
DROP TABLE IF EXISTS pdd_table;
CREATE TABLE pdd_table 
SELECT p.*, d.delivered_on
FROM patient_discharges p
JOIN deliveries d
ON p.id=d.patient_discharge_id;

/usr/local/lib/python2.7/dist-packages/sqlalchemy/engine/default.py:470: Warning: Unknown table 'pdd_table'
  cursor.execute(statement, parameters)


0 rows affected.
43625 rows affected.


[]

Define tables to be compared

In [4]:
tableA = 'patient_discharges'
tableB = 'births'

Define fields to compare tables by

In [5]:
tableA_fields = ['admitted_on','discharged_on'] #pdd table
#tableA_fields = ['date_of_birth'] #pdd table
tableB_fields = ['date_of_delivery'] #bc table

#comparisons = [
#    "date_of_birth = mothers_date_of_birth",
#    "delivered_on = date_of_delivery"
#]

Define field(s) to block by

In [6]:
tableA_blocks = ['hospital_id','sex_id']
tableB_blocks = ['hospital_id','sex_id']

### Block-wise linkage
Find all possible values for 1st blocking field ('hospital_id')

In [8]:
bl_stmt = 'SELECT T.blks FROM (' + \
'SELECT DISTINCT ' + tableA_blocks[0] + ' AS blks ' \
'FROM ' +  tableA + ' UNION ' + \
'SELECT DISTINCT ' + tableB_blocks[0] + ' AS blks ' \
'FROM ' +  tableB + ') as T ORDER BY blks;'

print(bl_stmt)
%sql PREPARE stmt_name FROM :bl_stmt;
block_list1 = %sql EXECUTE stmt_name;
print(block_list1)

0 rows affected.
0 rows affected.
12 rows affected.
0 rows affected.


Find all possible values for 2nd blocking field

In [9]:
bl_stmt = 'SELECT T.blks FROM (' + \
'SELECT DISTINCT ' + tableA_blocks[1] + ' AS blks ' \
'FROM ' +  tableA + ' UNION ' + \
'SELECT DISTINCT ' + tableB_blocks[1] + ' AS blks ' \
'FROM ' +  tableB + ') as T ORDER BY blks;'

print(bl_stmt)
%sql PREPARE stmt_name FROM :bl_stmt;
block_list2 = %sql EXECUTE stmt_name;
print(block_list2)

0 rows affected.
0 rows affected.
4 rows affected.
0 rows affected.


### Craft each block

In [13]:
stmt1 = 'CREATE TABLE blockA SELECT * FROM ' + tableA + ' a ' + \
'WHERE a.' + tableA_blocks[0] + '=' + repr(str(block_list1[0][0])) + \
' AND a.' + tableA_blocks[1] + '=' + repr(str(block_list2[1][0])) + ';'
print(stmt1)
stmt2 = 'CREATE TABLE blockB SELECT * FROM ' + tableB + ' b ' + \
'WHERE b.' + tableB_blocks[0] + '=' + repr(str(block_list1[0][0])) + \
' AND b.' + tableB_blocks[1] + '=' + repr(str(block_list2[1][0])) + ';'
print(stmt2)

SELECT * FROM patient_discharges a WHERE a.hospital_id='1' AND a.sex_id='1';
SELECT * FROM births b WHERE b.hospital_id='1' AND b.sex_id='1';


In [14]:
%%sql
DROP TABLE IF EXISTS blockA;
DROP TABLE IF EXISTS blockB;

SET @query = :stmt1;
PREPARE stmt_name FROM @query;
EXECUTE stmt_name;

SET @query = :stmt2;
PREPARE stmt_name FROM @query;
EXECUTE stmt_name;

0 rows affected.
0 rows affected.
2052 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
3756 rows affected.
0 rows affected.


UnicodeDecodeError: 'utf8' codec can't decode byte 0x94 in position 2: invalid start byte

### Cross join (and sub-sample) the fields-of-interest from both blocks

In [11]:
# Prepare strings for sql statement based on the field (column) names within each table
stmtA = ''.join(['a.%s, ' % (tableA_fields[n]) for n in xrange(len(tableA_fields))])
stmtB = ''.join(['b.%s, ' % (tableB_fields[n]) for n in xrange(len(tableB_fields))])
stmtB = stmtB[:-2] # Drop the final comma

In [20]:
cr_stmt = 'CREATE TABLE cross_table ' + 'SELECT a.id,' + stmtA + stmtB + ', b.id AS birth_id FROM blockA a CROSS JOIN blockB b;'
#'WHERE rand() <= .01;'
print(cr_stmt)

CREATE TABLE cross_table SELECT a.id,a.admitted_on, a.discharged_on, b.date_of_delivery, b.id AS birth_id FROM blockA a CROSS JOIN blockB b;


In [21]:
%%sql 
DROP TABLE IF EXISTS cross_table;

SET @query = :cr_stmt;
PREPARE stmt_name FROM @query;
EXECUTE stmt_name;

0 rows affected.
0 rows affected.
0 rows affected.
7707312 rows affected.


[]

In [ ]:
#RESUME HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! fix this so it gives me a boolean vector in python
# Dump sql table into python dataframe
cross_table = %sql SELECT * FROM cross_table
cross_table_df = cross_table.DataFrame()
print(cross_table_df)

In [159]:
print(cross_table_df[tableA_fields[0]])
print(cross_table_df[tableB_fields[0]])

0     1981-04-18
1     1981-04-18
2     1981-04-18
3     1981-04-18
4     1981-04-18
5     1981-04-18
6     1981-04-18
7     1972-05-29
8     1972-05-29
9     1972-05-29
10    1972-05-29
11    1972-05-29
12    1972-05-29
13    1972-05-29
14    2012-05-20
15    2012-05-20
16    2012-05-20
17    2012-05-20
18    2012-05-20
19    2012-05-20
20    2012-05-20
21    1986-04-29
22    1986-04-29
23    1986-04-29
24    1986-04-29
25    1986-04-29
26    1986-04-29
27    1986-04-29
28    2010-12-30
29    2010-12-30
         ...    
33    2010-12-30
34    2010-12-30
35    1977-02-24
36    1977-02-24
37    1977-02-24
38    1977-02-24
39    1977-02-24
40    1977-02-24
41    1977-02-24
42    2013-09-02
43    2013-09-02
44    2013-09-02
45    2013-09-02
46    2013-09-02
47    2013-09-02
48    2013-09-02
49    1976-10-23
50    1976-10-23
51    1976-10-23
52    1976-10-23
53    1976-10-23
54    1976-10-23
55    1976-10-23
56    1978-09-01
57    1978-09-01
58    1978-09-01
59    1978-09-01
60    1978-09-

In [154]:
print(cross_table_df[tableA_fields[0]]==cross_table_df[tableB_fields[0]])

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
      ...  
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
59    False
60    False
61    False
62    False
dtype: bool


### Create random subsets from both tables
Select X% of total data to estimate our probabilities

In [5]:
%%sql
DROP TABLE pdd_samp;
DROP TABLE bc_samp;

0 rows affected.
0 rows affected.


[]

In [6]:
%%sql
# Create sample PDD table (joined to deliveries table)
CREATE TABLE pdd_samp 
SELECT p.*, d.delivered_on
FROM (
    SELECT *  
    FROM patient_discharges) p
JOIN deliveries d
ON p.id=d.patient_discharge_id
WHERE rand() <= .01;

# Create sample BC table
CREATE TABLE bc_samp 
SELECT *  
FROM births b
WHERE rand() <= .01;

444 rows affected.
643 rows affected.


[]

### Estimate u probability for each field

In [7]:
# Loop through fields provided, and count number of exact matches from both tables
match_count = []
for fieldA, fieldB in zip(tableA_fields,tableB_fields):
    %sql SET @query = CONCAT('SELECT COUNT(*) FROM ', :tableA, ' a CROSS JOIN ', :tableB, ' b ', 'WHERE a.', :fieldA, '=b.', :fieldB, ';');
    %sql PREPARE stmt_name FROM @query;
    result = %sql EXECUTE stmt_name;
    match_count.append(float(result[0][0]))
    %sql DEALLOCATE PREPARE stmt_name;
print(match_count)

0 rows affected.
0 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
1 rows affected.
0 rows affected.
[23859.0, 33.0, 188.0]


In [8]:
# Double check loop output
obs_pair_cnt = %sql SELECT COUNT(*) FROM pdd_samp p CROSS JOIN bc_samp b WHERE p.hospital_id = b.hospital_id;
print(obs_pair_cnt)
obs_pair_cnt = %sql SELECT COUNT(*) FROM pdd_samp p CROSS JOIN bc_samp b WHERE p.date_of_birth = b.mothers_date_of_birth;
print(obs_pair_cnt)
obs_pair_cnt = %sql SELECT COUNT(*) FROM pdd_samp p CROSS JOIN bc_samp b WHERE p.delivered_on = b.date_of_delivery;
print(obs_pair_cnt)

1 rows affected.
+----------+
| COUNT(*) |
+----------+
|  23859   |
+----------+
1 rows affected.
+----------+
| COUNT(*) |
+----------+
|    33    |
+----------+
1 rows affected.
+----------+
| COUNT(*) |
+----------+
|   188    |
+----------+


In [9]:
# Find denominator (total number of possible pairs)
tot_pair_cnt = %sql SELECT COUNT(p.id) FROM pdd_samp p CROSS JOIN bc_samp b;
print(tot_pair_cnt)

# Calculate match probability based on number of observations/total pairs
u_probs = [x/tot_pair_cnt[0][0] for x in match_count]
print(u_probs)

# Set initial m probability values
m_probs = [.9] * len(u_probs)

1 rows affected.
+-------------+
| COUNT(p.id) |
+-------------+
|    240074   |
+-------------+
[0.09938185726067796, 0.00013745761723468598, 0.0007830918800036655]


In [11]:
1.0/365

0.0027397260273972603

In [21]:
type(tableA_fields[0])

str